In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:

from datasets import load_dataset
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.model_selection import train_test_split
from itertools import product

pd.set_option('display.max_columns', None)

In [ ]:
# this cell may take up to 30 seconds to run to convert the Hugginface Dataset class to Pandas DataFrame for better EDA

HF_SPOTIFY_DATASET_PATH = "maharshipandya/spotify-tracks-dataset"
# https://huggingface.co/datasets/maharshipandya/spotify-tracks-dataset

ds = load_dataset(HF_SPOTIFY_DATASET_PATH) #only has 'train' split as key
df = pd.DataFrame(ds["train"])

# Note: Used 10% of the data due to runtime crashing
# stratified based on genre
df, _ = train_test_split(
    df,
    train_size=0.1,
    stratify=df['track_genre'],
    random_state=42
)
df.head()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.68k [00:00<?, ?B/s]

dataset.csv:   0%|          | 0.00/20.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/114000 [00:00<?, ? examples/s]

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
42663,42663,3Ny9J2c1beLVf3lETydYan,Manger Cadavre?,AntiAutoAjuda,Fracasso,12,107106,False,0.209,0.801,1,-8.360,1,0.0840,0.000056,0.936,0.1340,0.506,174.701,4,grindcore
75582,75582,1ndhpiJgFIjK2TbQAEKrbM,Dan Gibson's Solitudes,Asian Spa,The Essence of Being,22,233777,False,0.412,0.224,4,-24.085,0,0.0377,0.871000,0.700,0.9670,0.229,95.030,4,new-age
34207,34207,3ZaTTeOT3K820yThVN6zHi,Nusrat Fateh Ali Khan;A1Melodymaster,Reformed,Mere Rashke Qamar,45,293944,False,0.654,0.850,5,-4.568,0,0.1570,0.038000,0.000,0.1840,0.532,104.859,4,folk
108836,108836,0wqqLC1IRgsgivUAmna9F4,Miguel Calo;Raúl Berón,Mi Cantar,Jamás Retornarás,16,150346,False,0.700,0.304,2,-7.406,0,0.0444,0.934000,0.290,0.0984,0.513,121.063,4,tango
74006,74006,1KMBAzA8X44WL5RfxkjSsu,Lagum,MEMÓRIAS (de onde eu nunca fui),FESTA JOVEM,49,112052,False,0.819,0.438,11,-10.611,1,0.3030,0.378000,0.000,0.3700,0.809,130.099,4,mpb


In [ ]:
#only use numeric values
df_numeric = df.select_dtypes(include=['int64', 'float64']).drop(["Unnamed: 0", "mode", "time_signature", "key"], axis=1)

In [ ]:
df_numeric.isna().sum()

,0
popularity,0
duration_ms,0
danceability,0
energy,0
loudness,0
speechiness,0
acousticness,0
instrumentalness,0
liveness,0
valence,0


no null values

In [ ]:
# stdev center data due to difference in scale
df_scaled = df_numeric.copy()
df_scaled = df_scaled / df_scaled.std()
df_scaled

,popularity,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
42663,0.537807,0.973268,1.215810,3.186565,-1.669541,0.798487,0.000167,3.055964,0.720467,1.953918,5.793999
75582,0.985980,2.124323,2.396717,0.891124,-4.809916,0.358368,2.603130,2.285443,5.199188,0.884283,3.151692
34207,2.016777,2.671058,3.804497,3.381498,-0.912256,1.492410,0.113569,0.000000,0.989297,2.054317,3.477673
108836,0.717076,1.366188,4.072091,1.209383,-1.479022,0.422057,2.791416,0.946826,0.529059,1.980949,4.015082
74006,2.196046,1.018212,4.764347,1.742466,-2.119079,2.880255,1.129716,0.000000,1.989348,3.123952,4.314763
...,...,...,...,...,...,...,...,...,...,...,...
21987,1.165249,1.983657,4.578194,2.657460,-1.040667,2.557058,0.430368,0.000000,0.580675,2.525420,3.115276
26447,1.030797,1.319909,2.262919,0.958754,-2.756740,0.383083,2.689801,0.000000,1.010804,0.973098,3.500424
18221,1.075615,1.891778,3.629979,2.880241,-1.712478,8.925940,2.465651,0.000000,4.586254,1.251125,3.708469
35169,2.016777,1.701416,3.792862,3.731583,-0.650242,0.562743,0.215184,0.002269,0.240335,3.672285,5.635568


# PCA and Analysis

In [ ]:
pca_U, pca_d, pca_V = np.linalg.svd(df_scaled)
pca_d

array([757.33706646, 174.19045061, 131.69548286, 117.92190598,
       109.76352295, 104.34351675,  98.63484719,  93.15163427,
        72.48787637,  60.5475444 ,  54.94119177])

In [ ]:
pca_U.shape

(11400, 11400)

In [ ]:
pca_V.shape

(11, 11)

In [ ]:
pca_d.shape

(11,)

# Cumulative Variability

In [ ]:
prop_var = np.square(pca_d) / sum(np.square(pca_d))
pd.DataFrame(
    {"PC": 1 + np.arange(0, prop_var.shape[0]),
     "variability_explained": prop_var.round(2),
     "cumulative_variability_explained": prop_var.cumsum().round(2)
     }).head(10)

,PC,variability_explained,cumulative_variability_explained
0,1,0.83,0.83
1,2,0.04,0.88
2,3,0.03,0.90
3,4,0.02,0.92
4,5,0.02,0.94
5,6,0.02,0.96
6,7,0.01,0.97
7,8,0.01,0.98
8,9,0.01,0.99
9,10,0.01,1.00


In [ ]:
# put the loadings for PC1 in a data frame
loadings1 = pd.DataFrame(
    {"features": df_scaled.columns,
     "pc1_loading": pca_V[0]
     })
# look at the 10 largest (absolute value) loadings for PC1 but print out the signed value
loadings1.reindex(loadings1["pc1_loading"].abs().sort_values(ascending=False).index) \
    .head(10)

,features,pc1_loading
10,tempo,-0.576850
2,danceability,-0.469695
3,energy,-0.365535
1,duration_ms,-0.295163
9,valence,-0.264847
4,loudness,0.223858
0,popularity,-0.211773
8,liveness,-0.162649
6,acousticness,-0.125983
5,speechiness,-0.117693


In [ ]:
# put the loadings for PC2 in a data frame
loadings2 = pd.DataFrame(
    {"features": df_scaled.columns,
     "pc2_loading": pca_V[1]
     })
# look at the 10 largest (absolute value) loadings for PC1 but print out the signed value
loadings2.reindex(loadings2["pc2_loading"].abs().sort_values(ascending=False).index) \
    .head(10)

,features,pc2_loading
4,loudness,0.612418
6,acousticness,-0.503345
3,energy,0.437492
7,instrumentalness,-0.327157
9,valence,0.205355
1,duration_ms,-0.118575
2,danceability,0.102047
5,speechiness,0.059122
0,popularity,-0.040280
8,liveness,0.034298


It seems that the spread is more explained by PC2 (the spread is mostly in the PC1 direction)

In [ ]:
# create the PCA-transformed dataset

# multiply the original data and the PCA loadings
pca_scaled_x = df_scaled@pca_V.T
# make the data frame pretty and easier to work with by
# changing the column names to PC1, PC2, etc
pca_scaled_x.columns = ["PC" + str(1 + col) for col in pca_scaled_x.columns]

# look at the object
pca_scaled_x.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11
42663,-6.789210,-0.253663,-2.117604,0.186645,0.847727,2.497747,1.370452,-1.386301,1.396032,-0.019269,-0.395727
75582,-6.787236,-4.326284,-0.668843,-2.987171,0.451645,-0.582440,-1.563590,-2.608199,-0.772326,0.693636,-0.104761
34207,-7.343762,1.346459,-0.011408,0.017685,0.001885,-1.169893,0.335268,0.424972,0.179843,0.096458,0.169308
108836,-6.633347,-1.476253,1.526196,0.489917,0.529849,0.742665,-0.142518,0.101071,-0.540168,-1.252847,-0.574572
74006,-8.235918,-0.010804,2.290066,-1.569261,-0.130622,-0.198052,1.014235,0.009481,0.009168,0.774612,-0.876559


In [ ]:
# plot PC1 vs PC2
px.scatter(pca_scaled_x, x="PC1", y="PC2",
           opacity=0.2, hover_name=df["track_genre"])

# Scree Plot
83% of the variability was explained by PC1.

In [ ]:
px.line(x=np.arange(prop_var.shape[0]),
        y=prop_var,
        labels={"x": "PC",
                "y": "Proportion explained"})

According to the scree plot, the elbow occurs at PC1, suggesting that a significant portion of the variability is captured by PC1

In [ ]:
# put the loadings for PC1 in a data frame
loadings1 = pd.DataFrame(
    {"category": df_scaled.columns,
     "pc1_loading": pca_V[0]
     })
# look at the 10 largest (absolute value) loadings for PC1 but print out the signed value
loadings1.reindex(loadings1["pc1_loading"].abs().sort_values(ascending=False).index) \
    .head(10)

,category,pc1_loading
10,tempo,-0.576850
2,danceability,-0.469695
3,energy,-0.365535
1,duration_ms,-0.295163
9,valence,-0.264847
4,loudness,0.223858
0,popularity,-0.211773
8,liveness,-0.162649
6,acousticness,-0.125983
5,speechiness,-0.117693


In [ ]:
# create the PCA-transformed dataset

# multiply the original data and the PCA loadings
pca_scaled_x = df_scaled@pca_V.T
# make the data frame pretty and easier to work with by
# changing the column names to PC1, PC2, etc
pca_scaled_x.columns = ["PC" + str(1 + col) for col in pca_scaled_x.columns]

# look at the object
pca_scaled_x.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11
42663,-6.789210,-0.253663,-2.117604,0.186645,0.847727,2.497747,1.370452,-1.386301,1.396032,-0.019269,-0.395727
75582,-6.787236,-4.326284,-0.668843,-2.987171,0.451645,-0.582440,-1.563590,-2.608199,-0.772326,0.693636,-0.104761
34207,-7.343762,1.346459,-0.011408,0.017685,0.001885,-1.169893,0.335268,0.424972,0.179843,0.096458,0.169308
108836,-6.633347,-1.476253,1.526196,0.489917,0.529849,0.742665,-0.142518,0.101071,-0.540168,-1.252847,-0.574572
74006,-8.235918,-0.010804,2.290066,-1.569261,-0.130622,-0.198052,1.014235,0.009481,0.009168,0.774612,-0.876559


We used PCA to analyze the structure of the data and PC1 and PC2 accounted for 88% of the variance in the data.  Points were labeled based on music genre, but the PC1 against PC2 graph did not reveal any significant clustering.